<a href="https://colab.research.google.com/github/tihitnag/image_generation/blob/main/final_data_loader122.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from typing import Tuple, List
import pandas as pd
from PIL import Image
from torch import zeros, Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from typing import Tuple
from torch import Generator as _DataGenerator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn

import torch.optim as optim


In [3]:
!apt-get install unrar
!pip install colorama
from google.colab import drive
drive.mount('/content/drive')


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Mounted at /content/drive


In [4]:

!apt-get install unrar

# Replace with the correct path to your file
rar_file_path = '/content/drive/MyDrive/imagefile.rar'
!mkdir -p /content/extracted_files
# Extract the .rar file
!unrar x {rar_file_path} /content/extracted_files

# Verify the extraction
!ls /content/extracted_files


Streaming output truncated to the last 5000 lines.
Extracting  /content/extracted_files/imagefile/abnormal/Image_7099.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_71.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_710.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_7100.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_7101.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_7102.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_7103.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_7104.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_7105.jpg      15%  OK 
Extracting  /content/extracted_files/imagefile/abnormal/Image_7106.jpg      15%  OK 
Extracting  

In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [6]:


# Constants
INPUT_PATH = '/path/to/osteoporosis_data'  # Root directory for your data
BATCH_SIZE = 32                            # Adjust as needed based on memory
IMAGE_SIZE = 64                           # Target image size
SEED = 42                                  # Random seed for reproducibility
NUM_CLASSES = 2                            # Number of classes (e.g., "normal" and "abnormal")
LABEL_TO_CLASS = {'normal': 0, 'abnormal': 1}  # Map class names to integers
                        # Number of workers for DataLoader

class OsteoporosisDataset(Dataset):
    def __init__(self, root_dir: str, data_transforms: transforms = None) -> None:
        """
        Custom dataset for loading osteoporosis data.
        """
        self.root_dir = root_dir
        self.class_names = ['normal', 'abnormal']
        self.image_paths, self.labels = self._load_images()
        self.transforms = self.get_data_transforms() if data_transforms is None else data_transforms

    def __len__(self) -> int:
        """
        Returns the total number of samples in the dataset.
        """
        return len(self.image_paths)

    def __getitem__(self, idx: int) -> Tuple[Tensor, int]:
        """
        Fetch a single sample from the dataset.
        """
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load image
        image = Image.open(image_path).convert('RGB')  # Open and convert image to RGB
        image = self.transforms(image)  # Apply transformations (resize, normalize, etc.)

        return image, label

    def _load_images(self) -> Tuple[List[str], List[int]]:
        """
        Load image file paths and corresponding labels.
        """
        image_paths, labels = [], []
        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(self.root_dir, class_name)
            print(f"Checking directory: {class_dir}")
            if not os.path.exists(class_dir):
                print(f"Directory not found: {class_dir}")
            for image_file in os.listdir(class_dir):
                if image_file.endswith('.png','.jpg'):  # Only process PNG files
                    image_paths.append(os.path.join(class_dir, image_file))
                    labels.append(label)
            class_dir = os.path.join(self.root_dir, class_name)

        return image_paths, labels

    def get_data_transforms(self) -> transforms.Compose:
        """
        Define and return the data transformations.
        """
        return transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),  # Resize to IMAGE_SIZE x IMAGE_SIZE
            transforms.ToTensor(),  # Convert image to PyTorch tensor
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
        ])


In [7]:


# Constants
INPUT_PATH = '/path/to/osteoporosis_data'  # Root directory for your data
BATCH_SIZE = 32                            # Adjust as needed based on memory
IMAGE_SIZE = 64                           # Target image size
SEED = 42                                  # Random seed for reproducibility
NUM_CLASSES = 2                            # Number of classes (e.g., "normal" and "abnormal")
LABEL_TO_CLASS = {'normal': 0, 'abnormal': 1}  # Map class names to integers
NUM_WORKERS = 2                           # Number of workers for DataLoader

class OsteoporosisDataset(Dataset):
    def __init__(self, root_dir: str, data_transforms: transforms = None) -> None:
        """
        Custom dataset for loading osteoporosis data.
        """
        self.root_dir = root_dir
        self.class_names = ['normal', 'abnormal']
        self.image_paths, self.labels = self._load_images()
        self.transforms = self.get_data_transforms() if data_transforms is None else data_transforms

    def __len__(self) -> int:
        """
        Returns the total number of samples in the dataset.
        """
        return len(self.image_paths)

    def __getitem__(self, idx: int) -> Tuple[Tensor, int]:
        """
        Fetch a single sample from the dataset.
        """
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load image
        image = Image.open(image_path).convert('RGB')  # Open and convert image to RGB
        image = self.transforms(image)  # Apply transformations (resize, normalize, etc.)

        return image, label

    def _load_images(self):
        image_paths = []
        labels = []

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(self.root_dir, class_name)
            if not os.path.exists(class_dir):
                print(f"Directory not found: {class_dir}")
                continue

            for image_file in os.listdir(class_dir):
                if image_file.endswith(('.png', '.jpg')):
                    image_paths.append(os.path.join(class_dir, image_file))
                    labels.append(label)

        print(f"Total images: {len(image_paths)}, Total labels: {len(labels)}")
        return image_paths, labels

    def get_data_transforms(self) -> transforms.Compose:
        """
        Define and return the data transformations.
        """
        return transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),  # Resize to IMAGE_SIZE x IMAGE_SIZE
            transforms.ToTensor(),  # Convert image to PyTorch tensor
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
        ])


In [8]:
from torch.utils.data import DataLoader, random_split
import torch
from typing import Tuple

def get_data_loader(root_dir: str) -> Tuple[DataLoader, DataLoader, DataLoader]:
    dataset = OsteoporosisDataset(root_dir)
    print(f"Total dataset size: {len(dataset)}")
    # Correctly compute dataset sizes
    train_size = int(0.6 * len(dataset))
    print(f"Total dataset size: {(train_size)}")
    val_size = int(0.2 * len(dataset))
    test_size = len(dataset) - train_size - val_size  # Adjust for rounding errors

    # Split the dataset
    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(SEED)  # Fix method name
    )

    # Create DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    return train_dataloader, val_dataloader, test_dataloader





In [9]:
if __name__ == '__main__':
    root_dir = "/content/extracted_files/imagefile"  # Ensure the path is a string

    train_dataloader, val_dataloader, test_dataloader = get_data_loader(root_dir)

    for images, labels in train_dataloader:
        print(f"Batch size: {images.size(0)}")
        print(f"Image shape: {images.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"traning image size):{len(train_dataloader)}")
        print(f"test image size):{len(test_dataloader)}")
        break


Total images: 11919, Total labels: 11919
Total dataset size: 11919
Total dataset size: 7151
Batch size: 32
Image shape: torch.Size([32, 3, 64, 64])
Labels shape: torch.Size([32])
traning image size):224
test image size):75


In [10]:
from colorama import Fore, Back, Style, init
import logging
from sys import stdout


# Prepare the colored formatter
init(autoreset=True)
terminal_colors = {"DEBUG": Fore.GREEN,  # "INFO": Fore.CYAN,
                   "WARNING": Fore.YELLOW, "ERROR": Fore.RED,
                   "CRITICAL": Fore.WHITE + Back.RED + Style.BRIGHT}
html_colors = {"DEBUG": 'BLUE',  # "INFO": Fore.CYAN,
               "WARNING": 'YELLOW', "ERROR": "ORANGE",
               "CRITICAL": "RED"}


class SingletonMeta(type):
    """
    Implementation of the Singleton pattern as Python metaclass.
    """
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(
                SingletonMeta, cls).__call__(*args, **kwargs)

        return cls._instances[cls]
        # else:
        # to run __init__ every time the class is called...
        #     cls._instances[cls].__init__(*args, **kwargs)


class ColoredFormatter(logging.Formatter):
    def format(self, record):
        msg = super().format(record)
        if record.levelname in terminal_colors:
            msg = terminal_colors[record.levelname] + msg + \
                  Fore.RESET + Back.RESET + Style.RESET_ALL
        return msg


class Logger(logging.Logger, metaclass=SingletonMeta):
    def __init__(self):
        fmt: str = "%(asctime)s %(levelname)8s >> %(message)s"
        # fmt: str = "%(asctime)s %(levelname)8s: %(name)18s >> %(message)s"
        date_format: str = "%m/%d/%Y %I:%M:%S %p"
        colored_formatter = ColoredFormatter(fmt, datefmt=date_format)
        bw_formatter = logging.Formatter(fmt, datefmt=date_format)

        # now we set the main / base logger called 'parent'
        super().__init__('hamgan-summary', logging.DEBUG)
        handler = logging.StreamHandler(stdout)
        handler.setLevel(logging.DEBUG)
        handler.setFormatter(colored_formatter)
        self.addHandler(handler)

    def _log(self, level: int, msg: object, args, **kwargs) -> None:
        """
        Log a message at a given level and always including the module's name
        from where the call was made.
        """
        logging.Logger.__dict__['_log'](self, level, msg, args, **kwargs)

In [11]:
#data generation process
import torch.nn as nn
from torch.cuda import is_available, device_count

# ---------------------
# General parameters
# ---------------------
SEED: int = 42
NUM_WORKERS: int = 4 * device_count()  # 4 * num_GPU as rule of thumb
nz: int = 128  # length of latent vector
ngf: int = 128  # depth of feature maps carried through the generator.
ndf: int = 64  # depth of feature maps propagated through the discriminator
# ngpu: int = 1
nc: int = 3  # number of color channels (for color images = 3)
# niter: int = NUM_EPOCHS BATCH_SIZE  # 300
n_dnn: int = 100
LEARNING_RATE: float = 0.0001
BETA_1: float = 0.5
NUM_CLASSES = 2
# ---------------------
# Training parameters
# ---------------------
BATCH_SIZE: int = 64 if IMAGE_SIZE == 64 else 32
NUM_EPOCHS: int = 100

class Generator64(nn.Module):
    def __init__(self):
        assert IMAGE_SIZE == 64, \
            f"This architecture is not suitable for IMAGE_SIZE = {IMAGE_SIZE}"
        super(Generator64, self).__init__()

        self.y_label = nn.Sequential(
            nn.Linear(NUM_CLASSES, n_dnn),  # 120, 1000
            nn.ReLU(True)
        )

        self.yz = nn.Sequential(
            nn.Linear(nz, 2 * nz),  # 100, 200
            nn.ReLU(True)
        )
        self.main = nn.Sequential(
            nn.ConvTranspose2d(n_dnn + 2 * nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Sigmoid()
            # state size. (nc) x 64 x 64
        )

    def forward(self, z, y):
        #print(f"z shape before yz: {z.shape}")
        z = self.yz(z)
        #print(f"z shape after yz: {z.shape}")
        y = self.y_label(y)
        #print(f"y shape after y_label: {y.shape}")
        inp = torch.cat([z, y], 1)
        #print(f"Concatenated input shape: {inp.shape}")
        inp = inp.view(-1, n_dnn + 2 * nz, 1, 1)
        output = self.main(inp)
        return output


class Discriminator64(nn.Module):
    def __init__(self):
        assert IMAGE_SIZE == 64, \
            f"This architecture is not suitable for IMAGE_SIZE = {IMAGE_SIZE}"
        super(Discriminator64, self).__init__()
        # self.ngpu = _ngpu
        self.y_label = nn.Sequential(
            nn.Linear(NUM_CLASSES, 64 * 64 * 1),
            nn.ReLU(True)
        )

        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc + 1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x, y):
    # Ensure y is correctly shaped before passing to Linear
          y = y.view(-1, NUM_CLASSES)  # Ensure y has shape [batch_size, NUM_CLASSES]
          #print(f"Label shape before Linear: {y.shape}")  # Debugging print
          y = self.y_label(y)  # Pass through Linear layer
          #print(f"Label shape after Linear: {y.shape}")  # Debugging print
          y = y.view(-1, 1, 64, 64)  # Reshape to match image dimensions
          #print(f"Label shape after reshaping: {y.shape}")  # Debugging print
          inp = torch.cat([x, y], 1)
          #print(f"Concatenated input shape: {inp.shape}")  # Debugging print
          output = self.main(inp)
          return output.view(-1, 1).squeeze(1)




In [12]:
# custom weights initialization called on generator and discriminator
def weights_init(m) -> callable:
    class_name = m.__class__.__name__
    if class_name.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif class_name.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [13]:
# import random
# from torch import Tensor, device
# from torchvision.utils import save_image
# import numpy as np
# import torchvision.utils as vutils
# import matplotlib.pyplot as plt

# #
# def seed_everything(seed=42):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     if torch.cuda.is_available():
#         torch.cuda.manual_seed_all(seed)
#         torch.backends.cudnn.deterministic = True
#         torch.backends.cudnn.benchmark = False
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# def __generate_random_noise() -> Tensor:
#     return torch.randn(BATCH_SIZE, nz, device=DEVICE)


# def __generate_random_labels() -> Tensor:
#     label = torch.zeros(BATCH_SIZE, NUM_CLASSES, device=DEVICE)
#     for i in range(BATCH_SIZE):
#         x = np.random.randint(0, NUM_CLASSES)
#         label[i][x] = 1
#     return label


# # random seeds for reproducibility
# seed_everything()
# # noise and labels from which images will be generated at checkpoint
# _checkpoint_noise = __generate_random_noise()
# _checkpoint_labels = __generate_random_labels()
# OUTPUT_PATH: str = 'output'
# MODELS_PATH: str = 'models'
# # we create directory to save the generated images and the trained models
# os.makedirs(OUTPUT_PATH, exist_ok=True)
# os.makedirs(MODELS_PATH, exist_ok=True)
# logger = Logger()



# def train_gan(
#         data_loader: DataLoader, use_cpu: bool = False,
#         save_best_model: bool = True, save_generated_images: bool = True,
#         verbose: bool = False, _freq: int = 5) -> Tuple[Generator64, Discriminator64]:

#     # initialize (with weights) generator and discriminator
#     net_g = Generator64().to(DEVICE)
#     net_g.apply(weights_init)
#     net_d = Discriminator64().to(DEVICE)
#     net_d.apply(weights_init)

#     # loss function and optimizers
#     criterion = nn.BCELoss()  # we are simply detecting whether it's real/fake

#     real_label = float(1)
#     fake_label = float(0)

#     # setup optimizer
#     optimizer_d = optim.Adam(
#         net_d.parameters(), lr=LEARNING_RATE, betas=(BETA_1, 0.999))
#     optimizer_g = optim.Adam(
#         net_g.parameters(), lr=LEARNING_RATE, betas=(BETA_1, 0.999))
#     d_error_epoch = []
#     g_error_epoch = []

#     for epoch in range(NUM_EPOCHS):
#         # we will start iterating each batch element
#         d_error_iter = 0
#         g_error_iter = 0
#         for i, data in enumerate(data_loader, 0):
#             # DISCRIMINATOR
#             # train with real
#             net_d.zero_grad()
#             real_cpu = data[0].to(DEVICE)
#             batch_size = real_cpu.size(0)
#             pathology_one_hot = data[1].to(DEVICE)
#              # Convert labels to one-hot encoding
#             pathology_one_hot = nn.functional.one_hot(data[1].to(DEVICE), num_classes=NUM_CLASSES).float()

#             label = torch.full((batch_size, ), real_label, device=DEVICE)

#             output = net_d(real_cpu, pathology_one_hot)
#             err_d_real = criterion(output, label)
#             err_d_real.backward()
#             # D_x = output.mean().item()

#             # train with fake
#             noise = torch.randn(batch_size, nz, device=DEVICE)
#             fake = net_g(noise, pathology_one_hot)
#             label.fill_(fake_label)
#             output = net_d(fake.detach(), pathology_one_hot)
#             err_d_fake = criterion(output, label)
#             err_d_fake.backward()
#             # D_G_z1 = output.mean().item()
#             err_d = err_d_real + err_d_fake
#             d_error_iter += err_d.item()
#             optimizer_d.step()

#             # GENERATOR
#             net_g.zero_grad()
#             label.fill_(real_label)  # fake labels are real for generator cost
#             output = net_d(fake, pathology_one_hot)
#             err_g = criterion(output, label)
#             g_error_iter += err_g.item()
#             err_g.backward()
#             # D_G_z2 = output.mean().item()
#             optimizer_g.step()


#             if (i + 1) % (BATCH_SIZE // 4) == 0 and verbose:
#                 _counter = f"Epoch [{epoch+1}/{NUM_EPOCHS}][{i+1}/{len(data_loader)}]"
#                 logger.debug(f"{_counter} --- Loss G: {err_g.item()}")
#                 logger.debug(f"{_counter} --- Loss D: {err_d.item()}")
#                 print(f"{_counter} --- Loss G: {err_g.item():.4f}, Loss D: {err_d.item():.4f}")

#         # Save generated images and models at certain frequency
#         if (epoch + 1) % _freq == 0:
#             with torch.no_grad():
#                 if save_generated_images:
#                     logger.debug(f"CHECKPOINT: saving some generated images at '{OUTPUT_PATH}' directory")
#                     checkpoint_images = net_g(_checkpoint_noise, _checkpoint_labels)
#                     save_image((checkpoint_images + 1) / 2,
#                                os.path.join(OUTPUT_PATH, f"epoch_{epoch + 1}.png"),
#                                nrow=8, normalize=True)

#             # Save models as checkpoint
#             if save_best_model:
#                 logger.debug(f"CHECKPOINT: saving the trained models at '{MODELS_PATH}' directory")
#                 torch.save(net_g.state_dict(), os.path.join(MODELS_PATH, "generator.pth"))
#                 torch.save(net_d.state_dict(), os.path.join(MODELS_PATH, "discriminator.pth"))

#         # Accumulate error for each epoch
#         d_error_epoch.append(d_error_iter)
#         g_error_epoch.append(g_error_iter)

#     _plot_losses(g_error_epoch, d_error_epoch)

#     # Save the trained generator and discriminator at the end of training
#     torch.save(net_g.state_dict(), os.path.join(MODELS_PATH, "generator.pth"))
#     torch.save(net_d.state_dict(), os.path.join(MODELS_PATH, "discriminator.pth"))

#     return net_g, net_d


# def _plot_losses(g_losses: List[float], d_losses: List[float], _show: bool = False) -> None:
#     plt.figure(figsize=(10, 5))
#     plt.title("Generator and Discriminator Loss During Training")
#     plt.plot(g_losses, label="Generator")
#     plt.plot(d_losses, label="Discriminator")
#     plt.xlabel("Epochs")
#     plt.ylabel("Loss")
#     plt.legend()

#     if _show:
#         plt.show()
#     plt.close()


# def _labels_to_tensor(labels: Tuple[str]) -> torch.Tensor:
#     return torch.tensor(tuple(LABEL_TO_CLASS[_l] for _l in labels))





In [ ]:
# train_gan(data_loader=train_dataloader, verbose=True, _freq=10)

In [ ]:
# _plot_losses(g_losses, d_error_epoch, _show=True)

In [14]:
import random
from torch import Tensor, device
from torchvision.utils import save_image
import numpy as np
import torchvision.utils as vutils
import matplotlib.pyplot as plt

#
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def __generate_random_noise() -> Tensor:
    return torch.randn(BATCH_SIZE, nz, device=DEVICE)


def __generate_random_labels() -> Tensor:
    label = torch.zeros(BATCH_SIZE, NUM_CLASSES, device=DEVICE)
    for i in range(BATCH_SIZE):
        x = np.random.randint(0, NUM_CLASSES)
        label[i][x] = 1
    return label


# random seeds for reproducibility
seed_everything()
# noise and labels from which images will be generated at checkpoint
_checkpoint_noise = __generate_random_noise()
_checkpoint_labels = __generate_random_labels()
OUTPUT_PATH: str = 'output'
MODELS_PATH: str = 'models'
# we create directory to save the generated images and the trained models
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)
logger = Logger()
def train_gan(
        data_loader: DataLoader, use_cpu: bool = False,
        save_best_model: bool = True, save_generated_images: bool = True,
        verbose: bool = False, _freq: int = 5) -> Tuple[Generator64, Discriminator64]:

    # Initialize (with weights) generator and discriminator
    net_g = Generator64().to(DEVICE)
    net_g.apply(weights_init)
    net_d = Discriminator64().to(DEVICE)
    net_d.apply(weights_init)

    # Use different learning rates
    optimizer_d = optim.Adam(net_d.parameters(), lr=0.0005, betas=(0.5, 0.999))
    optimizer_g = optim.Adam(net_g.parameters(), lr=0.0002, betas=(0.5, 0.999))

    criterion = nn.BCEWithLogitsLoss()


    # Start training loop
    for epoch in range(NUM_EPOCHS):
        for i, data in enumerate(data_loader):
            batch_size = data[0].size(0)

            # Train Discriminator
            net_d.zero_grad()
            real_images = data[0].to(DEVICE)
            labels = nn.functional.one_hot(data[1].to(DEVICE), NUM_CLASSES).float()

            # Add noise to real images for stability
            real_images = real_images + torch.randn_like(real_images) * 0.1

            # Label smoothing
            real_label = torch.full((batch_size,), 0.9, device=DEVICE)
            fake_label = torch.full((batch_size,), 0.0, device=DEVICE)  # Slightly better choice for stability

            # Forward pass for real images
            output_real = net_d(real_images, labels)
            d_loss_real = criterion(output_real, real_label)

            # Generate fake images
            noise = torch.randn(batch_size, nz, device=DEVICE)
            fake_images = net_g(noise, labels)
            output_fake = net_d(fake_images.detach(), labels)  # Detach fake images to avoid updating the generator here
            d_loss_fake = criterion(output_fake, fake_label)

            # Combine losses and backpropagate for Discriminator
            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            optimizer_d.step()
            torch.nn.utils.clip_grad_norm_(net_g.parameters(), max_norm=1.0)
            torch.nn.utils.clip_grad_norm_(net_d.parameters(), max_norm=1.0)


            # Train Generator multiple times (to try to fool the Discriminator more)
            for _ in range(2):  # Train generator 2 times for each discriminator update
                net_g.zero_grad()

                # Recompute fake images to ensure randomness
                fake_images = net_g(noise, labels)

                # Forward pass for fake images
                output_fake = net_d(fake_images, labels)
                g_loss = criterion(output_fake, real_label)  # Generator wants to fool discriminator
                g_loss.backward()
                optimizer_g.step()

            # Print losses (optional)
            if verbose and i % 50 == 0:
                print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Batch [{i+1}/{len(data_loader)}], '
                      f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')

            # Save images more frequently (every 100 batches)
            if save_generated_images and i % 100 == 0:
                with torch.no_grad():
                    test_images = net_g(_checkpoint_noise, _checkpoint_labels)
                    save_image(test_images * 0.5 + 0.5,
                             os.path.join(OUTPUT_PATH, f'epoch_{epoch+1}_batch_{i}.png'),
                             normalize=True)

            # Save best model (optional)
            if save_best_model and (epoch + 1) % _freq == 0:
                torch.save(net_g.state_dict(), os.path.join(MODELS_PATH, f'generator_epoch_{epoch+1}.pth'))
                torch.save(net_d.state_dict(), os.path.join(MODELS_PATH, f'discriminator_epoch_{epoch+1}.pth'))

    return net_g, net_d


In [15]:
train_gan(data_loader=train_dataloader, verbose=True, _freq=10)

Epoch [1/100], Batch [1/224], D_loss: 1.6168, G_loss: 0.6931
Epoch [1/100], Batch [51/224], D_loss: 1.7347, G_loss: 0.4133
Epoch [1/100], Batch [101/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [1/100], Batch [151/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [1/100], Batch [201/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [2/100], Batch [1/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [2/100], Batch [51/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [2/100], Batch [101/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [2/100], Batch [151/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [2/100], Batch [201/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [3/100], Batch [1/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [3/100], Batch [51/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [3/100], Batch [101/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [3/100], Batch [151/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [3/100], Batch [201/224], D_loss: 1.7265, G_loss: 0.4133
Epoch [4/100], Batch [1/224], D_loss: 1.7265, G_loss: 0.4133
Epo

KeyboardInterrupt: 

In [ ]:
train_gan(data_loader=train_dataloader, verbose=True, _freq=10)

In [ ]:
def plot_gan_losses(g_losses, d_losses, save_path='losses.png'):
    plt.figure(figsize=(12, 6))
    plt.plot(g_losses, label='Generator Loss', color='#2ecc71', linewidth=2)
    plt.plot(d_losses, label='Discriminator Loss', color='#e74c3c', linewidth=2)

    plt.grid(True, linestyle='--', alpha=0.7)
    plt.title('Generator and Discriminator Losses Over Time', pad=15, size=14)
    plt.xlabel('Epochs', size=12)
    plt.ylabel('Loss', size=12)

    window = 10
    if len(g_losses) > window:
        g_avg = np.convolve(g_losses, np.ones(window)/window, mode='valid')
        d_avg = np.convolve(d_losses, np.ones(window)/window, mode='valid')
        plt.plot(g_avg, '--', color='#27ae60', alpha=0.5, label='Generator Moving Avg')
        plt.plot(d_avg, '--', color='#c0392b', alpha=0.5, label='Discriminator Moving Avg')

    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
generator, discriminator = train_gan(data_loader=train_dataloader, _freq=5)
plot_gan_losses(g_losses, d_losses, save_path='gan_losses_plot.png')

In [ ]:
plot_gan_losses(g_losses, d_losses, save_path='gan_losses_plot.png')

In [ ]:
LEARNING_RATE_D = 0.0002
LEARNING_RATE_G = 0.0001
def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

def generate_random_noise(batch_size=BATCH_SIZE):
    return torch.randn(batch_size, nz, device=DEVICE)

def generate_random_labels(batch_size=BATCH_SIZE):
    label = torch.zeros(batch_size, NUM_CLASSES, device=DEVICE)
    for i in range(batch_size):
        x = np.random.randint(0, NUM_CLASSES)
        label[i][x] = 1
    return label

def plot_gan_losses(g_losses, d_losses, save_path='losses.png'):
    plt.figure(figsize=(12, 6))
    plt.plot(g_losses, label='Generator Loss', color='#2ecc71', linewidth=2)
    plt.plot(d_losses, label='Discriminator Loss', color='#e74c3c', linewidth=2)

    plt.grid(True, linestyle='--', alpha=0.7)
    plt.title('Generator and Discriminator Losses Over Time', pad=15, size=14)
    plt.xlabel('Epochs', size=12)
    plt.ylabel('Loss', size=12)

    window = 10
    if len(g_losses) > window:
        g_avg = np.convolve(g_losses, np.ones(window)/window, mode='valid')
        d_avg = np.convolve(d_losses, np.ones(window)/window, mode='valid')
        plt.plot(g_avg, '--', color='#27ae60', alpha=0.5, label='Generator Moving Avg')
        plt.plot(d_avg, '--', color='#c0392b', alpha=0.5, label='Discriminator Moving Avg')

    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def train_gan(data_loader, generator, discriminator, num_epochs=NUM_EPOCHS):
    # Initialize networks
    generator = generator.to(DEVICE)
    discriminator = discriminator.to(DEVICE)
    generator.apply(weights_init)
    discriminator.apply(weights_init)

    # Loss and optimizers
    criterion = nn.BCELoss()
    optimizer_d = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE_D, betas=(BETA_1, 0.999))
    optimizer_g = optim.Adam(generator.parameters(), lr=LEARNING_RATE_G, betas=(BETA_1, 0.999))

    # Lists to store losses
    g_losses = []
    d_losses = []

    # Checkpoint noise and labels
    fixed_noise = generate_random_noise()
    fixed_labels = generate_random_labels()

    for epoch in range(num_epochs):
        d_loss_epoch = 0
        g_loss_epoch = 0

        for i, (real_images, labels) in enumerate(data_loader):
            batch_size = real_images.size(0)
            real_images = real_images.to(DEVICE)
            labels = nn.functional.one_hot(labels, NUM_CLASSES).float().to(DEVICE)

            # Train Discriminator
            discriminator.zero_grad()
            label_real = torch.full((batch_size,), 0.9, device=DEVICE)
            label_fake = torch.full((batch_size,), 0.0, device=DEVICE)

            output_real = discriminator(real_images, labels)
            d_loss_real = criterion(output_real, label_real)

            noise = generate_random_noise(batch_size)
            fake_images = generator(noise, labels)
            output_fake = discriminator(fake_images.detach(), labels)
            d_loss_fake = criterion(output_fake, label_fake)

            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            optimizer_d.step()

            # Train Generator
            generator.zero_grad()
            output_fake = discriminator(fake_images, labels)
            g_loss = criterion(output_fake, label_real)
            g_loss.backward()
            optimizer_g.step()

            d_loss_epoch += d_loss.item()
            g_loss_epoch += g_loss.item()

            if i % 50 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(data_loader)}], '
                      f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')

        # Save images and losses
        if (epoch + 1) % 5 == 0:
            with torch.no_grad():
                fake_images = generator(fixed_noise, fixed_labels)
                save_image(fake_images * 0.5 + 0.5,
                          os.path.join(OUTPUT_PATH, f'epoch_{epoch+1}.png'),
                          normalize=True)

            g_losses.append(g_loss_epoch/len(data_loader))
            d_losses.append(d_loss_epoch/len(data_loader))
            plot_gan_losses(g_losses, d_losses)

            # Save models
            torch.save(generator.state_dict(),
                      os.path.join(MODELS_PATH, f'generator_epoch_{epoch+1}.pth'))
            torch.save(discriminator.state_dict(),
                      os.path.join(MODELS_PATH, f'discriminator_epoch_{epoch+1}.pth'))

    return generator, discriminator, g_losses, d_losses

if __name__ == "__main__":
    # Create directories
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    os.makedirs(MODELS_PATH, exist_ok=True)



    # Set random seed
    seed_everything()

In [ ]:
generator, discriminator, g_losses, d_losses = train_gan(
    data_loader=train_dataloader,
    generator=Generator(nz=nz, num_classes=NUM_CLASSES),
    discriminator=Discriminator(num_classes=NUM_CLASSES),
    num_epochs=NUM_EPOCHS,
    verbose=True,
    _freq=10
)


In [ ]:
def visualize_progress(generator, test_noise, test_labels, epoch):
    generator.eval()
    with torch.no_grad():
        fake = generator(test_noise, test_labels)
        plt.figure(figsize=(10,10))
        plt.imshow(np.transpose(vutils.make_grid(fake.cpu(), padding=2, normalize=True), (1,2,0)), cmap='gray')
        plt.axis('off')
        plt.title(f'Generated Images - Epoch {epoch}')
        plt.savefig(f'generated_epoch_{epoch}.png')
        plt.close()

# In training loop, add:
if epoch % 10 == 0:
    visualize_progress(net_g, _checkpoint_noise, _checkpoint_labels, epoch)
    print(f"\nEpoch [{epoch}/{NUM_EPOCHS}]")
    print(f"Generator Loss: {g_loss.item():.4f}")
    print(f"Discriminator Loss: {d_loss.item():.4f}")